In [1]:
# AXILiteS
# 0x0000 : Control signals
#          bit 0  - ap_start (Read/Write/COH)
#          bit 1  - ap_done (Read/COR)
#          bit 2  - ap_idle (Read)
#          bit 3  - ap_ready (Read)
#          bit 7  - auto_restart (Read/Write)
#          others - reserved
# 0x0004 : Global Interrupt Enable Register
#          bit 0  - Global Interrupt Enable (Read/Write)
#          others - reserved
# 0x0008 : IP Interrupt Enable Register (Read/Write)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x000c : IP Interrupt Status Register (Read/TOW)
#          bit 0  - Channel 0 (ap_done)
#          bit 1  - Channel 1 (ap_ready)
#          others - reserved
# 0x1000 : Data signal of num_symbols
#          bit 31~0 - num_symbols[31:0] (Read/Write)
# 0x1004 : reserved
# 0x0400 ~
# 0x07ff : Memory 'parent_V' (255 * 32b)
#          Word n : bit [31:0] - parent_V[n]
# 0x0800 ~
# 0x0bff : Memory 'left_V' (255 * 32b)
#          Word n : bit [31:0] - left_V[n]
# 0x0c00 ~
# 0x0fff : Memory 'right_V' (255 * 32b)
#          Word n : bit [31:0] - right_V[n]
# 0x1100 ~
# 0x11ff : Memory 'length_histogram_V' (64 * 32b)
#          Word n : bit [31:0] - length_histogram_V[n]
# (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)

from pynq import Overlay

overlay = Overlay("../bitstream/compute-bit-length-test.bit")
overlay.download()

compute_bit_len = overlay.compute_bit_length_0

# set symbol numbers
compute_bit_len.write(0x1000, 6)

# given tree
# pos : 0 parent : 1
#         lchild : 102
#         rchild : 98
# pos : 1 parent : 2
#         lchild : 99
#         rchild : 4294967295
# pos : 2 parent : 4
#         lchild : 97
#         rchild : 4294967295
# pos : 3 parent : 4
#         lchild : 100
#         rchild : 101
# pos : 4 parent : 0
#         lchild : 4294967295
#         rchild : 4294967295

tree_nodes = [
    { 'p': 1, 'l': 102, 'r':  98 },
    { 'p': 2, 'l':  99, 'r':  -1 },
    { 'p': 4, 'l':  97, 'r':  -1 },
    { 'p': 4, 'l': 100, 'r': 101 },
    { 'p': 0, 'l':  -1, 'r':  -1 },
]

for idx, node in enumerate(tree_nodes):
    # write parent
    compute_bit_len.write(0x0400 + 4*idx, node['p'])
    # write left & right child
    compute_bit_len.write(0x0800 + 4*idx, node['l'])
    compute_bit_len.write(0x0c00 + 4*idx, node['r'])

# start
compute_bit_len.write(0x0000, 1)

from time import sleep

sleep(1)

for i in range(10):
    bit_len = compute_bit_len.read(0x1100 + 4*i)
    if (not bit_len == 0):
        print('L : ' + str(i) + ' N : ' + str(bit_len))

L : 2 N : 3
L : 3 N : 1
L : 4 N : 2
